<a href="https://colab.research.google.com/github/Vrindakishoree/genai-hackathon/blob/main/yoga_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import files
files.upload()  # upload kaggle.json here


Saving kaggle (3).json to kaggle (3).json


{'kaggle (3).json': b'{"username":"vrindakishoree","key":"1f5a9257ec01e5c8bb2b22782af2a1a4"}'}

In [9]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [10]:
from google.colab import files
import json, os

# 1. Upload kaggle.json
uploaded = files.upload()  # select your kaggle.json

# 2. Read it
creds = json.loads(next(iter(uploaded.values())).decode())

# 3. Set environment variables
os.environ['KAGGLE_USERNAME'] = creds['username']
os.environ['KAGGLE_KEY']      = creds['key']

# 4. Also save it to ~/.kaggle (for completeness)
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle credentials set:")
print("   KAGGLE_USERNAME =", os.environ['KAGGLE_USERNAME'])


Saving kaggle (3).json to kaggle (3) (1).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
✅ Kaggle credentials set:
   KAGGLE_USERNAME = vrindakishoree


In [11]:
!kaggle datasets download -d pulaksarmah/yoga-videos
!unzip -q yoga-videos.zip -d yoga_videos


Dataset URL: https://www.kaggle.com/datasets/pulaksarmah/yoga-videos
License(s): CC0-1.0
 99% 880M/893M [00:07<00:00, 115MB/s] 
100% 893M/893M [00:07<00:00, 127MB/s]


In [12]:
import os

vids = [f for f in os.listdir('yoga_videos') if f.endswith('.mp4')]
print(f"✅ Found {len(vids)} videos.")
print("Sample:", vids[:5])


✅ Found 0 videos.
Sample: []


In [13]:
import os

for root, dirs, files in os.walk("yoga_videos"):
    print(f"📁 Folder: {root} — {len(files)} files")
    if files:
        print("  📄 Sample files:", files[:3])


📁 Folder: yoga_videos — 0 files
📁 Folder: yoga_videos/dataset — 0 files
📁 Folder: yoga_videos/dataset/test — 0 files
📁 Folder: yoga_videos/dataset/test/Trikasana — 3 files
  📄 Sample files: ['video 12.mp4', 'video 13.mp4', 'video 11.mp4']
📁 Folder: yoga_videos/dataset/test/Vrikshasana — 3 files
  📄 Sample files: ['video 14.mp4', 'video 12.mp4', 'video 13.mp4']
📁 Folder: yoga_videos/dataset/test/Padamasana — 3 files
  📄 Sample files: ['video 14.mp4', 'video 12.mp4', 'video 13.mp4']
📁 Folder: yoga_videos/dataset/test/Tadasana — 3 files
  📄 Sample files: ['video 14.mp4', 'video 13.mp4', 'video 15.mp4']
📁 Folder: yoga_videos/dataset/test/Bhujasana — 3 files
  📄 Sample files: ['video 14.mp4', 'video 13.mp4', 'video 15.mp4']
📁 Folder: yoga_videos/dataset/train — 0 files
📁 Folder: yoga_videos/dataset/train/Trikasana — 10 files
  📄 Sample files: ['video 4.mp4', 'video 5.mp4', 'video 2.mp4']
📁 Folder: yoga_videos/dataset/train/Vrikshasana — 11 files
  📄 Sample files: ['video 4.mp4', 'video 5.mp

In [14]:
!pip install mediapipe opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling op

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

def extract_keypoints_from_video(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    keypoints = []
    frame_count = 0

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(image_rgb)

        if result.pose_landmarks:
            kp = [[lm.x, lm.y, lm.z] for lm in result.pose_landmarks.landmark]
            keypoints.append(np.array(kp).flatten())  # 33 landmarks * 3 coords = 99

        frame_count += 1

    cap.release()
    if len(keypoints) == max_frames:
        return np.array(keypoints).flatten()  # 30 frames × 99 = 2970 features
    else:
        return None


In [2]:
X, y = [], []

base_dir = "yoga_videos"  # Change if your videos are inside subfolders

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.mp4'):
            label = os.path.basename(root)  # folder name = pose label
            video_path = os.path.join(root, file)
            features = extract_keypoints_from_video(video_path)
            if features is not None:
                X.append(features)
                y.append(label)

print("✅ Extracted:", len(X), "samples")


✅ Extracted: 63 samples


In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Encode labels and scale features
le = LabelEncoder()
y_encoded = le.fit_transform(y)
X = np.array(X)
X_scaled = StandardScaler().fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=16)


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.3016 - loss: 2.1284 - val_accuracy: 0.6000 - val_loss: 1.6431
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.7016 - loss: 0.9255 - val_accuracy: 0.6000 - val_loss: 1.6725
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7703 - loss: 0.6684 - val_accuracy: 0.6000 - val_loss: 2.0222
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8156 - loss: 0.5606 - val_accuracy: 0.6000 - val_loss: 1.9304
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.8359 - loss: 0.4873 - val_accuracy: 0.6000 - val_loss: 1.8379
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9109 - loss: 0.1862 - val_accuracy: 0.6000 - val_loss: 1.8552
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9797 - loss: 0.0828 - val_accuracy: 0.6000 - val_loss: 1.8656
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9438 - loss: 0.1179 - val_accuracy: 0.7000 - val_loss: 1.7707
Epoch 9/20

In [4]:
model.save("yoga_pose_classifier.h5")


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
model.save('/content/drive/MyDrive/yoga_pose_classifier.h5')


In [7]:
!cp -r yoga_videos /content/drive/MyDrive/yoga_videos


In [8]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle

# Mount Google Drive (if not already)
from google.colab import drive
drive.mount('/content/drive')

# Load trained model
model = load_model('/content/drive/MyDrive/yoga_pose_classifier.h5')

# Load label encoder if you saved it (optional)
# Or reinitialize with same classes used during training:
le = LabelEncoder()
le.classes_ = np.array(['Tree', 'Warrior', 'DownwardDog'])  # use your actual labels


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install mediapipe


In [10]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm


In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!ls /content/drive/MyDrive


'Aadhar Card.pdf'	       'Rank Card.pdf'
'Attendance Survey.gsheet'     'Screen Recording 2025-06-22 183535.mp4'
'Colab Notebooks'	       'Share Vrind.docx'
'College ID.pdf'	       'Untitled form (1).gform'
'Document from KVK.pdf'        'Untitled form.gform'
'Google AI Studio'	        Vrindares.docx
'Google AI Studio (1)'	       'Vrind documentation'
 iktara.mp3		        yoga_pose_classifier.h5
'IMG-20230331-WA0003 (1).jpg'   yoga_pose_dataset.csv
 IMG-20230331-WA0003.jpg        yoga_videos
'Income Certificate.pdf'


!ls '/content/drive/MyDrive/yoga_videos'


In [14]:
DATASET_DIR = '/content/drive/MyDrive/your/path/to/yoga_data'


In [15]:
DATASET_DIR = '/content/drive/MyDrive/yoga_videos/yoga_data'


In [16]:
!ls '/content/drive/MyDrive/your/actual/path/yoga_data'


ls: cannot access '/content/drive/MyDrive/your/actual/path/yoga_data': No such file or directory


In [17]:
DATASET_DIR = '/content/drive/MyDrive/yoga_videos'


In [18]:
!ls '/content/drive/MyDrive/yoga_videos'


dataset  yoga_videos


In [19]:
!pip install mediapipe


In [ ]:
!pip install numpy==1.26.4 mediapipe==0.10.11 --force-reinstall --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.

In [20]:
import numpy as np
import mediapipe as mp
print("✅ All set! numpy =", np.__version__)


✅ All set! numpy = 1.26.4


In [21]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm

# Set your dataset path
DATASET_DIR = '/content/drive/MyDrive/yoga_videos'
MAX_FRAMES = 30

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

X, y = [], []

def extract_keypoints(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []
    count = 0

    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        if result.pose_landmarks:
            keypoints = [[lm.x, lm.y, lm.z] for lm in result.pose_landmarks.landmark]
            flat = np.array(keypoints).flatten()
            if flat.shape[0] == 99:
                keypoints_list.append(flat)
                count += 1

    cap.release()

    while len(keypoints_list) < max_frames:
        keypoints_list.append(np.zeros(99))

    if keypoints_list:
        return np.array(keypoints_list).flatten()
    return None

# Loop through dataset
for label in os.listdir(DATASET_DIR):
    label_path = os.path.join(DATASET_DIR, label)
    if not os.path.isdir(label_path):
        continue

    for video in tqdm(os.listdir(label_path), desc=f"Processing {label}"):
        if video.endswith('.mp4'):
            video_path = os.path.join(label_path, video)
            features = extract_keypoints(video_path)
            if features is not None:
                X.append(features)
                y.append(label)

# Convert to numpy and save
X = np.array(X)
y = np.array(y)

np.save('/content/X_keypoints.npy', X)
np.save('/content/y_labels.npy', y)

print("✅ Keypoint extraction complete!")
print("X shape:", X.shape)
print("y shape:", y.shape)


Processing yoga_videos: 100%|██████████| 1/1 [00:00<00:00, 11397.57it/s]

✅ Keypoint extraction complete!
X shape: (0,)
y shape: (0,)


In [24]:
DATASET_DIR = '/content/drive/MyDrive/test_yoga_data'


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Simulate small dataset
X = np.random.rand(10, 2970)  # 10 videos × 30 frames × 99 keypoints
y = np.array(['Tree', 'DownwardDog', 'Warrior', 'Tree', 'Warrior', 'Tree', 'DownwardDog', 'Warrior', 'Tree', 'Warrior'])

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_scaled, y_train)

# Evaluate
acc = clf.score(X_test_scaled, y_test)
print(f"✅ Model trained! Accuracy: {acc:.2f}")


✅ Model trained! Accuracy: 0.00


In [26]:
import joblib
np.save('label_classes.npy', le.classes_)
joblib.dump(clf, 'pose_classifier.pkl')
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [27]:
# Run this if not already defined earlier
def extract_keypoints(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []
    count = 0

    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        if result.pose_landmarks:
            keypoints = [[lm.x, lm.y, lm.z] for lm in result.pose_landmarks.landmark]
            flat = np.array(keypoints).flatten()
            if flat.shape[0] == 99:
                keypoints_list.append(flat)
                count += 1

    cap.release()

    while len(keypoints_list) < max_frames:
        keypoints_list.append(np.zeros(99))

    if keypoints_list:
        return np.array(keypoints_list).flatten()
    return None


In [28]:
# Load model and scaler
clf = joblib.load('pose_classifier.pkl')
scaler = joblib.load('scaler.pkl')
label_classes = np.load('label_classes.npy')

# Predict on new video
features = extract_keypoints("tree_pose.mp4")
if features is not None:
    features_scaled = scaler.transform([features])
    prediction = clf.predict(features_scaled)[0]
    print("🧘 Predicted Pose:", label_classes[prediction])
else:
    print("⚠️ Pose could not be detected in video.")


🧘 Predicted Pose: Warrior


In [29]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm

# Initialize MediaPipe pose detector
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

# Path to your dataset (update this if needed)
DATASET_DIR = '/content/drive/MyDrive/yoga_videos'

# Hyperparameter
MAX_FRAMES = 30

# Containers for keypoints and labels
X = []
y = []


In [31]:
def extract_keypoints(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []
    count = 0

    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(rgb)

        if result.pose_landmarks:
            kp = [[lm.x, lm.y, lm.z] for lm in result.pose_landmarks.landmark]
            flat = np.array(kp).flatten()
            if flat.shape[0] == 99:
                keypoints_list.append(flat)
                count += 1

    cap.release()

    # Pad with zeros if frames are too few
    while len(keypoints_list) < max_frames:
        keypoints_list.append(np.zeros(99))

    if keypoints_list:
        return np.array(keypoints_list).flatten()
    else:
        return None


In [32]:
# Loop through dataset folders and videos
for label in os.listdir(DATASET_DIR):
    label_path = os.path.join(DATASET_DIR, label)
    if not os.path.isdir(label_path): continue

    print(f"🔍 Processing class: {label}")
    for video in tqdm(os.listdir(label_path), desc=f"⏳ {label}"):
        if video.endswith('.mp4'):
            video_path = os.path.join(label_path, video)
            features = extract_keypoints(video_path)

            if features is not None:
                X.append(features)
                y.append(label)
            else:
                print(f"⚠️ Skipped (no pose detected): {video}")

X = np.array(X)
y = np.array(y)

np.save('X_keypoints.npy', X)
np.save('y_labels.npy', y)

print("✅ Keypoint extraction complete!")
print("✅ Total samples:", len(X))
print("✅ X shape:", X.shape)
print("✅ y shape:", y.shape)
print("✅ Classes:", set(y))


🔍 Processing class: dataset


⏳ dataset: 100%|██████████| 2/2 [00:00<00:00, 11667.05it/s]


🔍 Processing class: yoga_videos


⏳ yoga_videos: 100%|██████████| 1/1 [00:00<00:00, 5714.31it/s]

✅ Keypoint extraction complete!
✅ Total samples: 0
✅ X shape: (0,)
✅ y shape: (0,)
✅ Classes: set()


In [33]:
import os

for root, dirs, files in os.walk('/content/drive/MyDrive/yoga_videos'):
    level = root.replace('/content/drive/MyDrive/yoga_videos', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


yoga_videos/
  dataset/
    test/
      Bhujasana/
        video 13.mp4
        video 14.mp4
        video 15.mp4
      Padamasana/
        video 12.mp4
        video 13.mp4
        video 14.mp4
      Tadasana/
        video 13.mp4
        video 14.mp4
        video 15.mp4
      Trikasana/
        video 11.mp4
        video 12.mp4
        video 13.mp4
      Vrikshasana/
        video 12.mp4
        video 13.mp4
        video 14.mp4
    train/
      Bhujasana/
        video 1.mp4
        video 10.mp4
        video 11.mp4
        video 12.mp4
        video 2.mp4
        video 3.mp4
        video 4.mp4
        video 5.mp4
        video 6.mp4
        video 7.mp4
        video 8.mp4
        video 9.mp4
      Padamasana/
        video 1.mp4
        video 10.mp4
        video 11.mp4
        video 2.mp4
        video 3.mp4
        video 4.mp4
        video 5.mp4
        video 6.mp4
        video 7.mp4
        video 8.mp4
        video 9.mp4
      Tadasana/
        video 1.mp4
        video 10.

In [34]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import joblib

# Setup MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Paths
TRAIN_DIR = '/content/drive/MyDrive/yoga_videos/dataset/train'
MAX_FRAMES = 30

X = []
y = []

def extract_keypoints(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []

    frame_count = 0
    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(image_rgb)

        if result.pose_landmarks:
            landmarks = result.pose_landmarks.landmark
            keypoints = [lm.x for lm in landmarks] + [lm.y for lm in landmarks] + [lm.z for lm in landmarks]
            keypoints_list.append(keypoints)

        frame_count += 1

    cap.release()

    if keypoints_list:
        return np.mean(keypoints_list, axis=0)
    else:
        return None

# Loop through train videos
print("🔄 Extracting features from training videos...")
for label in tqdm(os.listdir(TRAIN_DIR)):
    label_path = os.path.join(TRAIN_DIR, label)
    if not os.path.isdir(label_path): continue

    for file in os.listdir(label_path):
        if file.endswith('.mp4'):
            path = os.path.join(label_path, file)
            keypoints = extract_keypoints(path, MAX_FRAMES)
            if keypoints is not None:
                X.append(keypoints)
                y.append(label)

X = np.array(X)
y = np.array(y)

print(f"\n✅ Extracted {len(X)} samples.")
print("X shape:", X.shape)
print("Classes:", set(y))


🔄 Extracting features from training videos...


100%|██████████| 5/5 [01:31<00:00, 18.21s/it]


✅ Extracted 56 samples.
X shape: (56, 99)
Classes: {'Padamasana', 'Trikasana', 'Vrikshasana', 'Bhujasana', 'Tadasana'}


In [35]:
# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_scaled, y_train)

# Evaluate
y_pred = clf.predict(X_test_scaled)
print("\n📊 Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Save model, scaler and label encoder
joblib.dump(clf, 'pose_classifier.pkl')
joblib.dump(scaler, 'scaler.pkl')
np.save('label_classes.npy', le.classes_)



📊 Classification Report:

              precision    recall  f1-score   support

   Bhujasana       1.00      1.00      1.00         3
  Padamasana       1.00      1.00      1.00         3
    Tadasana       0.50      1.00      0.67         2
   Trikasana       1.00      1.00      1.00         2
 Vrikshasana       0.00      0.00      0.00         2

    accuracy                           0.83        12
   macro avg       0.70      0.80      0.73        12
weighted avg       0.75      0.83      0.78        12



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [36]:
# Load saved model and scaler
clf = joblib.load('pose_classifier.pkl')
scaler = joblib.load('scaler.pkl')
label_classes = np.load('label_classes.npy')

# Test with any video
test_video = '/content/drive/MyDrive/yoga_videos/dataset/test/Padamasana/video 12.mp4'
features = extract_keypoints(test_video)
if features is not None:
    features_scaled = scaler.transform([features])
    prediction = clf.predict(features_scaled)[0]
    print("🧘 Predicted Pose:", label_classes[prediction])
else:
    print("⚠️ No pose detected in the video.")


🧘 Predicted Pose: Padamasana


In [37]:
# 1️⃣ Install downloader
!pip install yt-dlp --quiet

# 2️⃣ Download the video
!yt-dlp -f "mp4" -o "yt_tree3min.mp4" "https://www.youtube.com/watch?v=tSivbr4f2sc"


[youtube] Extracting URL: https://www.youtube.com/watch?v=tSivbr4f2sc
[youtube] tSivbr4f2sc: Downloading webpage
[youtube] tSivbr4f2sc: Downloading tv client config
[youtube] tSivbr4f2sc: Downloading tv player API JSON
[youtube] tSivbr4f2sc: Downloading ios player API JSON
[youtube] tSivbr4f2sc: Downloading m3u8 information
[info] tSivbr4f2sc: Downloading 1 format(s): 18
[download] yt_tree3min.mp4 has already been downloaded
[download] 100% of   12.82MiB


In [38]:
features = extract_keypoints("yt_tree3min.mp4", max_frames=30)

if features is not None:
    features_scaled = scaler.transform([features])
    prediction = clf.predict(features_scaled)[0]
    print("🧘 Predicted Pose:", label_classes[prediction])
else:
    print("⚠️ Pose not detected.")


🧘 Predicted Pose: Trikasana


In [39]:
import joblib
joblib.dump(clf, "final_pose_classifier.pkl")
joblib.dump(scaler, "scaler.pkl")
np.save("label_classes.npy", label_classes)


In [40]:
import os

# Show uploaded files in Colab
os.listdir('/content')


['.config',
 'pose_classifier.pkl',
 'scaler.pkl',
 'final_pose_classifier.pkl',
 'label_classes.npy',
 'tree_pose.mp4',
 'yoga_pose_classifier.h5',
 'X_keypoints.npy',
 'y_labels.npy',
 'yoga_videos',
 'kaggle (3) (1).json',
 'yoga-videos.zip',
 'drive',
 'kaggle (3).json',
 'yt_tree3min.mp4',
 'sample_data']

In [42]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

def extract_keypoints_from_video(video_path, max_frames=30):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []
    frame_count = 0

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)
        if result.pose_landmarks:
            landmarks = result.pose_landmarks.landmark
            keypoints = []
            for lm in landmarks:
                keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])
            keypoints_list.append(keypoints)
            frame_count += 1
    cap.release()

    if keypoints_list:
        return np.array(keypoints_list)
    else:
        return None

# Load files
model = joblib.load('final_pose_classifier.pkl')
scaler = joblib.load('scaler.pkl')
label_classes = np.load('label_classes.npy', allow_pickle=True)

# Extract keypoints from your video
video_path = "/content/Padmasana (Lotus Pose)  Steps and Benefits  Kaivalyadhama Yoga Institute.mp4"
features = extract_keypoints_from_video(video_path)

if features is not None:
    # Average across frames
    features_mean = np.mean(features, axis=0).reshape(1, -1)
    features_scaled = scaler.transform(features_mean)
    prediction = model.predict(features_scaled)[0]
    print("🧘 Predicted Pose:", label_classes[prediction])
else:
    print("⚠️ Pose not detected.")


⚠️ Pose not detected.


In [43]:
!pip install mediapipe==0.10.9


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
  Attempting uninstall: mediapipe
    Found existing installation: mediapipe 0.10.21
    Uninstalling mediapipe-0.10.21:
      Successfully uninstalled mediapipe-0.10.21
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompati

In [16]:
label_classes = np.load("label_classes.npy")


In [38]:
# STEP 0: Upload your video
from google.colab import files
uploaded = files.upload()

# Replace this with your uploaded video filename
video_path = list(uploaded.keys())[0]

# STEP 1: Extract features
video_features = extract_keypoints_from_video(video_path, max_frames=30)

# STEP 2: Reshape and average
if video_features is not None and not np.isnan(video_features).any():
    try:
        video_features = np.array(video_features).reshape(30, 99)  # 30 frames, 99 features per frame
        averaged_features = np.mean(video_features, axis=0).reshape(1, -1)  # Shape: (1, 99)

        # STEP 3: Load model, scaler, and labels if not already loaded
        import joblib
        import numpy as np

        model = joblib.load("pose_classifier.pkl")
        scaler = joblib.load("scaler.pkl")
        label_classes = np.load("label_classes.npy")  # This is a list, not a LabelEncoder

        # STEP 4: Scale and predict
        features_scaled = scaler.transform(averaged_features)
        prediction = model.predict(features_scaled)[0]
        print("🧘 Predicted Pose:", label_classes[prediction])

    except Exception as e:
        print("❌ Error during reshaping or prediction:", str(e))
else:
    print("⚠️ No valid keypoints detected or data is invalid.")


Saving Padmasana (Lotus Pose)  Steps and Benefits  Kaivalyadhama Yoga Institute.mp4 to Padmasana (Lotus Pose)  Steps and Benefits  Kaivalyadhama Yoga Institute (5).mp4
✅ Pose detected in 30 frame(s).
🧘 Predicted Pose: Padamasana


In [39]:
# STEP 0: Upload your video
from google.colab import files
uploaded = files.upload()

# Replace this with your uploaded video filename
video_path = list(uploaded.keys())[0]

# STEP 1: Extract features
video_features = extract_keypoints_from_video(video_path, max_frames=30)

# STEP 2: Reshape and average
if video_features is not None and not np.isnan(video_features).any():
    try:
        video_features = np.array(video_features).reshape(30, 99)  # 30 frames, 99 features per frame
        averaged_features = np.mean(video_features, axis=0).reshape(1, -1)  # Shape: (1, 99)

        # STEP 3: Load model, scaler, and labels if not already loaded
        import joblib
        import numpy as np

        model = joblib.load("pose_classifier.pkl")
        scaler = joblib.load("scaler.pkl")
        label_classes = np.load("label_classes.npy")  # This is a list, not a LabelEncoder

        # STEP 4: Scale and predict
        features_scaled = scaler.transform(averaged_features)
        prediction = model.predict(features_scaled)[0]
        print("🧘 Predicted Pose:", label_classes[prediction])

    except Exception as e:
        print("❌ Error during reshaping or prediction:", str(e))
else:
    print("⚠️ No valid keypoints detected or data is invalid.")


Saving Yoga Asana - Bhujangasana (Cobra Pose) - Relieves Stress & Fatigue.mp4 to Yoga Asana - Bhujangasana (Cobra Pose) - Relieves Stress & Fatigue (3).mp4
✅ Pose detected in 30 frame(s).
🧘 Predicted Pose: Bhujasana
